In [ ]:
# Install required packages (uncomment if needed)
!pip install pandas numpy openpyxl

In [ ]:
# Upload your CSV file
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import warnings
from pathlib import Path

pd.set_option('display.max_columns', 39)

In [ ]:
# Helper Functions
def create_a_filter(df, column, StringIncluded):
    return df[column].str.contains(StringIncluded, na=False, case=False)

def add_comment_to_flagged_items(check_items, comment):
    check_items['CKEY'] = check_items['CKEY'].fillna(comment)

def create_a_string_length_filter(df, column, desired_length, str_to_replace='; '):
    return df[column].str.replace(str_to_replace, '', regex=False).str.replace(u'\u200e', '', regex=False).replace(np.nan, 'x' * desired_length).apply(len).apply(lambda x: x % desired_length == 0)

def flag_items(df, column, string_values, comment, optional_ignore_str=False):
    if optional_ignore_str:
        item_filter = create_a_filter(df, column, string_values) & ~create_a_filter(df, column, optional_ignore_str)
    else:
        item_filter = create_a_filter(df, column, string_values)
    items_to_check = df[item_filter].copy()
    add_comment_to_flagged_items(items_to_check, comment)
    return items_to_check

def flag_incorrect_length(df, column, length, comment, str_to_replace='; '):
    items_filter = create_a_string_length_filter(df, column, length, str_to_replace)
    items_check = df[~items_filter].copy()
    add_comment_to_flagged_items(items_check, comment)
    return items_check

def flag_items_with_2_strings(df, column, first_string, second_string, comment):
    item_filter = create_a_filter(df, column, first_string) & create_a_filter(df, column, second_string)
    items_to_check = df[item_filter].copy()
    add_comment_to_flagged_items(items_to_check, comment)
    return items_to_check

def flag_missing_data(df, column, comment, includedtype=False, excludedType=False):
    if includedtype:
        items_filter = df[column].isna() & df['Type'].isin(includedtype)
    elif excludedType:
        items_filter = df[column].isna() & ~df['Type'].isin(excludedType)
    else:
        items_filter = df[column].isna()
    items_check = df.loc[items_filter].copy()
    add_comment_to_flagged_items(items_check, comment)
    return items_check

def create_the_df(file_name):
    df = pd.read_csv(file_name,
                     dtype={'ISBN10': str, 'ISBN13': str, 'ISSN': str, 'EISSN': str, 'LCN': str,'Library Note': str, 'Student Note': str, 'Importance': str},
                     parse_dates=['Date Added', 'Last Published'])
    df = df.sort_values(by=["List Appearance"])
    return df

def remove_library_and_training_lists(df, unnecessary_lists):
    unnecessary_lists_filter = create_a_filter(df, 'List Appearance', unnecessary_lists)
    df.drop(index=df[unnecessary_lists_filter].index, inplace=True)
    return df

def create_academic_bookmarks_df(df, staff_names):
    academic_filter = df['Added By'].isin(staff_names)
    academic_df = df.loc[~academic_filter].copy()
    academic_df.dropna(subset="Added By", inplace=True)
    return academic_df

def create_paragraphs_df(df):
    paragraph_filter = df.dropna(how='all', subset=['Title', 'Chapter/Article Title', 'Author(s)', 'Type'])
    return df[~df.index.isin(paragraph_filter.index)]

def remove_paragraph_items(df):
    return df.dropna(how='all', subset=['Title', 'Chapter/Article Title', 'Author(s)', 'Type'])

def create_archives_df(df):
    return df.loc[create_a_filter(df, "Web Address", "https://archive.org")]

def create_TADCS_df(df):
    return df.loc[df['TADC Request Status'].isin(['LIVE', 'REFERRED', 'EXPIRED'])]

def create_urls_and_DOIs_df(df):
    return df.dropna(how='all', subset=['DOI', 'Web Address', 'Primary Web Address', 'Secondary Web Address'])

def flag_rejected_withdrawn_TADCS(df):
    TADCs = df.dropna(subset=['TADC Request Status'])
    TADC_check_df = TADCs.loc[~TADCs['TADC Request Status'].isin(['LIVE', 'REFERRED', 'EXPIRED'])].copy()
    add_comment_to_flagged_items(TADC_check_df, "Checked rejected TADC")
    return TADC_check_df

def sort_URLS_from_longest_to_shortest(df):
    url_length = df['Web Address'].str.split('; ').replace(np.nan, 'Missing').apply(lambda x: max(x, key=len)).apply(len)
    df["URL Length"] = url_length
    return df.sort_values("URL Length", ascending=False)

def flag_print_chapters_articles(print_df):
    TADC_missing_filt = print_df["Type"].isin(['Chapter', 'Article'])
    TADC_missing_check = print_df.loc[TADC_missing_filt].copy()
    add_comment_to_flagged_items(TADC_missing_check, "Missing TADC?")
    return TADC_missing_check

def flag_bad_urls(df):
    bad_urls = ["summon", "primo", "coursebank", "UQL.eblib", "learn.uq", "proxy.openathens", "ezproxy"]
    items_to_check = [flag_items(df, "Web Address", x, f"{x} URL") for x in bad_urls]
    alma_urls_check = flag_items(df, "Web Address", "alma", "alma URL", optional_ignore_str="permalink")
    ebsco_urls_check = flag_items(df, "Web Address", "ebsco", "Incorrect EBSCO URL", optional_ignore_str="custid=s1097571|search.ebscohost.com%2Flogin.aspx%3Fauthtype%3Dip%2Cuid%26profile%3Dbsi")
    gruyter_urls_check = flag_items_with_2_strings(df, "Web Address", "gruyter", "/product/", "incorrect De Gruyter URL")
    espace_urls_check = flag_items_with_2_strings(df, "Web Address", "espace", "pdf", "espace pdf URL")
    return pd.concat(items_to_check + [alma_urls_check, ebsco_urls_check, gruyter_urls_check, espace_urls_check])

def flag_bad_DOIS(df, column, comment):
    items_filt = (~df[column].str.startswith("10.", na=False)) & (df[column].notna())
    items_to_check = df.loc[items_filt].copy()
    add_comment_to_flagged_items(items_to_check, comment)
    return items_to_check

def check_for_missing_auth(cell, vendors, delim="; "):
    if isinstance(cell, str):
        cell_list = cell.lower().split(delim)
        return any(any(v in x for v in vendors) and "openathens" not in x and "resolver.library" not in x for x in cell_list)
    return False

def flag_missing_auth(df, column, comment, vendors):
    item_filter = df[column].apply(lambda x: check_for_missing_auth(x, vendors))
    items_to_check = df[item_filter].copy()
    add_comment_to_flagged_items(items_to_check, comment)
    return items_to_check

def create_vendors_df(df, column, vendors):
    pattern = '|'.join(vendors) + "|ebscohost|proquest"
    item_filter = df[column].str.contains(pattern, case=False, na=False)
    vendor_urls = df[item_filter].copy()
    dois = df.loc[df["DOI"].notna()]
    return pd.concat([vendor_urls, dois]).drop_duplicates()

def create_no_view_online_df(df, comment):
    view_online_filter = df['Online Resource Source'].isna() & df['Importance'].isin(["Recommended", "Further"]) & ~df['TADC Request Status'].isin(['LIVE', 'REFERRED', 'EXPIRED']) & ~create_a_filter(df, "Web Address", "https://archive.org")
    view_online_check = df.loc[view_online_filter].copy()
    add_comment_to_flagged_items(view_online_check, comment)
    return view_online_check

def flag_open_url(df, column, comment):
    item_filter = df[column] == "Open Url"
    items_to_check = df[item_filter].copy()
    add_comment_to_flagged_items(items_to_check, comment)
    return items_to_check

def flag_author_name_format(df, column, comment):
    item_filter = df[column].str.contains(",", na=False) & ~df[column].str.contains(";", na=False)
    items_to_check = df[item_filter].copy()
    add_comment_to_flagged_items(items_to_check, comment)
    return items_to_check

In [ ]:
# === Main Execution ===
file_name = list(uploaded.keys())[0]

In [ ]:
# Define your variables
unnecessary_lists = ' test| Library| Resources| Toolkit|TEST0018'
staff_names = ['Dowe,Rachel', 'Buteeva,Rita', 'Deutsch,Corinna', 'Hull,Natalie', 'Owens,Kia', 'Lau,Ella', 'McDouall,Alex']
vendors = ["oxford", "degruyter", "wiley", "jstor", "springer", "muse.jhu", "duke", "informit", "springer", "link.gale", "clinicalkey", "academic.oup", "brill", "sagepub", "fulcrum", "tandfonline", "loebclassics", "cabidigital", "taylorfrancis", "bloomsbury", "doi.org", "ovid", "factiva", "austlit", "sciencedirect", "vitalsource", "heinonline", "manchesterhive", "knovel", "alexanderstreet", "r2library"]


In [ ]:
# Load and process the data
df = create_the_df(file_name)
df = remove_library_and_training_lists(df, unnecessary_lists)
academic_df = create_academic_bookmarks_df(df, staff_names)
paragraph_df = create_paragraphs_df(df)
df = remove_paragraph_items(df)
archive_df = create_archives_df(df)
TADCS_df = create_TADCS_df(df)
urls_and_DOIs_df = create_urls_and_DOIs_df(df)
online_df = pd.concat([TADCS_df, urls_and_DOIs_df], axis='rows', sort=False).drop_duplicates()
print_df = df[~df.index.isin(online_df.index)]
vendors_df = create_vendors_df(urls_and_DOIs_df, "Web Address", vendors)
url_length_df = sort_URLS_from_longest_to_shortest(df)
df = df.drop(columns=["URL Length"])
view_online_button = create_no_view_online_df(urls_and_DOIs_df, "View Online Button?")

In [ ]:
# Flag issues
check_dataframes = [
      flag_missing_auth(urls_and_DOIs_df, "Web Address", "Missing openathen auth", vendors),
      flag_missing_data(vendors_df, "LCN", "Add LCN to eBook?", includedtype=["Book"]),
      flag_missing_data(print_df, "LCN", "No access - Missing LCN/URL"),
      flag_print_chapters_articles(print_df),
      flag_missing_data(df, "Date of Publication", "Date of pub", excludedType=['Webpage', 'Website', 'Journal']),
      flag_items(archive_df, "Online Resource Web Address", "archive.org", "Check archive.org access"),
      flag_items(print_df, "Publisher", "springer", "Check for Springer ebook"),
      flag_bad_urls(df),
      flag_items(df, 'Online Resource Web Address', 'ezproxy', "View Online button glitching"),
      flag_rejected_withdrawn_TADCS(df),
      flag_bad_DOIS(df, "DOI", "Check DOI"),
      flag_incorrect_length(df, "ISBN10", 10, "ISBN10"),
      flag_incorrect_length(df, "ISBN13", 13, "ISBN13"),
      flag_incorrect_length(df.assign(checking=df["ISSN"].str.replace("-", "").str.replace("–", "")), "checking", 8, "ISSN"),
      flag_incorrect_length(df.assign(checking=df["EISSN"].str.replace("-", "").str.replace("–", "")), "checking", 8, "EISSN"),
      pd.concat([
          flag_incorrect_length(df, "LCN", 18, "LCN"),
          flag_items(df, "LCN", ";", "2 LCNS")
      ]).drop_duplicates(subset="Item Link"),
      flag_items(df, "Student Note", "requested|Note for students|blackboard", "Check Student Note"),
      flag_items(df, "Library Note", "read |can |help|please|pls|blackboard", "Check Library Note", optional_ignore_str="You can remove this|We have added|we have ordered|please delete this bookmark if"),
      flag_open_url(df, "Online Resource Source", "Default Open URL"),
      flag_author_name_format (df, "Author(s)", "Check Author Field")
]

In [ ]:
# Combine all flagged items
check_items_df = pd.concat(check_dataframes, sort=False)

In [ ]:

# Create Excel workbook
def create_multiple_sheets_excel(dataframes_to_export, excel_sheet_titles):
    wb = Workbook()
    for d, s in zip(dataframes_to_export, excel_sheet_titles):
        ws = wb.create_sheet(title=s)
        for r in dataframe_to_rows(d, index=False, header=True):
            ws.append(r)
    if 'Sheet' in wb.sheetnames:
        wb.remove(wb['Sheet'])
    return wb

In [ ]:
# Save workbook
output_filename = "output.xlsx"
workbook = create_multiple_sheets_excel(
    [check_items_df, df, paragraph_df, url_length_df, view_online_button],
    ["Flagged", "Format Check", "Paragraphs", "URL Lengths", "View Online Button"]
)
workbook.save(output_filename)

In [ ]:
# Download the file
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>